In [1]:
import sys
import os
sys.path.append(r"D:\CursorProj\Chem-Ontology-Constructor")
os.environ["PROJECT_ROOT"] = "D:\\CursorProj\\Chem-Ontology-Constructor\\"

In [2]:
from autology_constructor.assertions import chemonto_with_entities_assertions, chemonto_with_elements_assertions, chemonto_with_data_properties_assertions, chemonto_with_object_properties_assertions
from autology_constructor.modules import ChemOntology
from autology_constructor.ontology_merge import merge_ontology
from src.ontology.preprocess import create_metadata_properties


d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import dspy

lm = dspy.LM('openai/gpt-4o', temperature=0, max_tokens=10000)
dspy.configure(lm=lm)
teacher = dspy.LM("openai/gpt-4o",temperature=1)

In [4]:
import json

def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
    return data

data = read_json_file(r'D:\CursorProj\Chem-Ontology-Constructor\tests\Carbon\result\result_final.json')
print(data[12])

{'Title': "'Post-combustion CO2 capture and recovery by pure activated\\nmethyldiethanolamine in crossflow membrane contactors having coated\\nhollow fibers\\n'", 'Abstract': "'A B S T R A C T\\nGas absorption and stripping using membrane contactors is one of a number of carbon capture and storage\\ntechnologies being investigated for separating CO2 from flue gas. Energy consumption in CO2 absorption-\\nstripping employing amine-containing aqueous absorbent solutions is strongly influenced by demanding strip-\\nping conditions involving higher temperatures. The utility of using pure methyldiethanolamine (MDEA) acti-\\nvated by piperazine as a reactive absorbent was studied using a compact hollow fiber device for the absorber as\\nwell as the stripper. Water needed for reactive absorption of CO2 in tertiary amine MDEA was obtained from\\nsimulated humidified flue gas stream which is saturated with moisture in actual practice. This study investigated\\nalso the absorption and stripping p

In [5]:
data = [{"content": "title: " + dict["Title"] + "\nabstract: " + dict["Abstract"], "source": dict["Title"], "file_path": ""} for dict in data]
data

[{'content': "title: 'Comparison study of activators performance for MDEA solution of acid\\ngases capturing from natural gas: Simulation-based on a real plant\\n'\nabstract: 'Abstract \\nEnergy requirements and environmental regulations are the main limiting \\nissues behind the continuous developments of amine absorption processes. \\nSteady state simulation has been performed using the Aspen HYSYS program \\nversion 8.8 to determine the effects of adding various concentrations of \\nactivators up to 10% with maintaining the constancy of the entire amine \\nstrength of 45%. The present investigation addresses the performance of using \\npiperazine/MDEA and sulfolane/MDEA on the absorption process in terms of \\nenergy consumption and acid gases loading. Four scenarios have been analyzed \\nunder the same operating conditions, 5% activator/ 40% MDEA has shown \\nbetter performance superior to other scenarios regarding the balance between \\nenergy consumption and absorption capacity o

In [6]:
for index, chunk in enumerate(data[-1:]):
# index = 4
# chunk = data[index]
    print(f"第{index+1}个文献")
    test_context = chunk["content"] + chunk["content"][-1]  # 添加最后一个字符
    source = chunk["source"]
    print(f"前30个字符: {test_context[:30]}")
    extract = ChemOntology()
    result = extract(test_context)
    create_metadata_properties()
    merge_ontology(
        result.ontology_entities, 
        result.ontology_elements, 
        result.ontology_data_properties, 
        result.ontology_object_properties,
        source,
        ""
    )

第1个文献
前30个字符: title: 'Surface Tension of N‑M
Creating SourcedInformation class...
Creating content property...
Creating source property...
Creating file_path property...
Creating type property...
Creating property property...
Creating has_information property...


In [6]:
import json
from owlready2 import get_ontology
import owlready2

def save_classes_hierarchy(ontology_path, output_path):
    # 读取本体
    ontology = get_ontology(ontology_path).load()
    
    classes_hierarchy = {}

    # 获取所有类及其父类和子类
    for cls in ontology.classes():
        parent_classes = list(cls.is_a)
        child_classes = list(cls.subclasses())

        classes_hierarchy[cls.name] = {
            "parents": [parent.name for parent in parent_classes if isinstance(parent, owlready2.entity.ThingClass)], # 修改这里，只处理类对象
            "children": [child.name for child in child_classes]
        }
    
    # 保存为json文件
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(classes_hierarchy, json_file, ensure_ascii=False, indent=4)


In [ ]:

# 调用函数，指定本体路径和输出文件路径
save_classes_hierarchy(r'D:\CursorProj\Chem-Ontology-Constructor\data\ontology\carbon.owl', r'D:\CursorProj\Chem-Ontology-Constructor\tests\Carbon\result\classes_hierarchy.json')


In [7]:
save_classes_hierarchy(r"F:\MainFolder\Document\WeChat Files\wxid_esmy188qpaz312\FileStorage\File\2025-03\chem_ontology.owl",r'D:\CursorProj\Chem-Ontology-Constructor\tests\Carbon\result\test2.json')

SourcedInformation
[owl.Thing]
[]
----------------------------------------------------------
fischer-tropsch_process
[owl.Thing, meta\.has_information.value(meta\.sourcedinformation1)]
[]
----------------------------------------------------------
enzymatic_ft-type_reaction
[classes\.nitrogenase_enzyme, meta\.has_information.value(meta\.sourcedinformation2), meta\.has_information.value(meta\.sourcedinformation8), meta\.has_information.value(meta\.sourcedinformation9), data_properties\.reaction_condition.value('ambient conditions'), meta\.has_information.value(meta\.sourcedinformation10), data_properties\.reducing_equivalent.value('H+ and e−'), object_properties\.is_catalyzed_by.some(classes\.nitrogenase_enzyme | classes\.metallocluster | classes\.synthetic_mimic), meta\.has_information.value(meta\.sourcedinformation11)]
[]
----------------------------------------------------------
nitrogenase_enzyme
[owl.Thing, meta\.has_information.value(meta\.sourcedinformation3), object_properties\.h

In [6]:
for index, chunk in enumerate(data[2:3]):
# index = 4
# chunk = data[index]
    print(f"第{index+1}个文献")
    test_context = chunk["content"]
    source = chunk["source"]
    print(f"前30个字符: {test_context[:30]}")
    temp = chemonto_with_entities_assertions(test_context)
    elements = chemonto_with_elements_assertions(test_context, temp)
    # data_properties =   chemonto_with_data_properties_assertions(test_context, temp)
    # object_properties = chemonto_with_object_properties_assertions(test_context, temp)
    create_metadata_properties()
    merge_ontology(
        temp.ontology_entities, 
        elements.ontology_elements, 
        None,# data_properties.ontology_data_properties, 
        None,# object_properties.ontology_object_properties,
        source,
        ""
    )

第1个文献
前30个字符: title: 'Effect of\xa0acidic pr
entities
Entities:
  - Name: n-methyldiethanolamine(MDEA)
    Information: N-methyldiethanolamine is the primary amine used in the study for CO2 and H2S separation from natural gas.
  - Name: acidic_oxidative_products
    Information: Acidic oxidative products are formed from the oxidation of MDEA, which include various acids.
  - Name: glycolic_acid
    Information: Glycolic acid is one of the acidic products from MDEA oxidation.
  - Name: oxalic_acid
    Information: Oxalic acid is another acidic product from MDEA oxidation.
  - Name: formic_acid
    Information: Formic acid is an acidic product that acts as a promoter in the desorption process.
  - Name: acetic_acid
    Information: Acetic acid is an acidic product from MDEA oxidation.
  - Name: thiosulfuric_acid
    Information: Thiosulfuric acid is an acidic product that acts as a promoter in the desorption process.
  - Name: sulfuric_acid
    Information: Sulfuric acid is an acidic pro

2025/02/21 21:15:59 INFO dspy.primitives.assertions: SuggestionFailed: The ontology does not explicitly demonstrate its applicability across different chemical subdomains, and there is no indication of how it integrates with other ontologies. To improve, the ontology could include examples or references to its use in other chemical contexts and provide a framework for integration with existing ontologies.
2025/02/21 21:16:14 INFO dspy.primitives.assertions: SuggestionFailed: The ontology does not explicitly demonstrate its applicability across different chemical subdomains, and there is no indication of how it integrates with other ontologies. To improve, the ontology could include examples or references to its use in other chemical contexts and provide a framework for integration with existing ontologies.


entities
Entities:
  - Name: n-methyldiethanolamine(MDEA)
    Information: N-methyldiethanolamine is the primary amine used in the study for CO2 and H2S separation from natural gas.
  - Name: acidic_oxidative_products
    Information: Acidic oxidative products are formed from the oxidation of MDEA, which include various acids.
  - Name: glycolic_acid
    Information: Glycolic acid is one of the acidic products from MDEA oxidation.
  - Name: oxalic_acid
    Information: Oxalic acid is another acidic product from MDEA oxidation.
  - Name: formic_acid
    Information: Formic acid is an acidic product that acts as a promoter in the desorption process.
  - Name: acetic_acid
    Information: Acetic acid is an acidic product from MDEA oxidation.
  - Name: thiosulfuric_acid
    Information: Thiosulfuric acid is an acidic product that acts as a promoter in the desorption process.
  - Name: sulfuric_acid
    Information: Sulfuric acid is an acidic product from MDEA oxidation.
  - Name: phosphori

KeyboardInterrupt: 

In [12]:
from config.settings import ONTOLOGY_CONFIG
onto = ONTOLOGY_CONFIG["ontology"]
infos = list(onto.individuals())
for info in infos:
    print("type: ", info.type)
    print("content: ", info.content)
    print("source: ", info.source)
    print("property: ", info.property)
    print("---\n")


type:  ['entity']
content:  ['Aryl-extended calix[4]pyrroles are macrocycles containing four pyrrole rings connected through their pyrrolic 2- and 5-positions by tetra-substituted sp3 carbon atoms (meso-substituents).']
source:  ['https://doi.org/10.1021/acs.accounts.2c00839']
property:  []
---

type:  ['entity']
content:  ['Super aryl-extended calix[4]pyrroles are derivatives of aryl-extended calix[4]pyrroles with enhanced binding properties for charged and neutral polar guests.']
source:  ['https://doi.org/10.1021/acs.accounts.2c00839']
property:  []
---

type:  ['entity']
content:  ['Meso-octamethyl calix[4]pyrrole is an outstanding receptor for anion binding introduced by Sessler in 1996.']
source:  ['https://doi.org/10.1021/acs.accounts.2c00839']
property:  []
---

type:  ['entity']
content:  ["The cone conformation of 'two-wall' αα-aryl-extended calix[4]pyrroles features an aromatic cleft with a polar binding site defined by four converging pyrrole NHs."]
source:  ['https://doi.o

In [6]:
from owlready2 import *
ontology = get_ontology(r'F:\MainFolder\Document\WeChat Files\wxid_esmy188qpaz312\FileStorage\File\2025-03\chem_ontology.owl').load()

classone = ontology['carbon-rich_resources']


In [7]:
classone.has_information()

AttributeError: 'NoneType' object has no attribute 'has_information'